In [69]:
import simpy
import numpy as np
import matplotlib.pyplot as plt

In [70]:
# Anylogic Bank
# Create a function
# * Input: 1 parameter (lam) characterizing the customer arrival rate per minute
#          In the original exercise, lam = 0.75

# * Output: The mean flow time for customers

def sim_flow_time(lam):
    class BankSimulation:
        def __init__(self, sim_time, arrival_rate):
        ## Simulation parameters  ##
        # atm 
        # cs = cashier station
        #
            self.sim_time = sim_time
            self.arrival_rate = arrival_rate
            self.num_atm = 1
            self.num_cashier = 5


        ## Simpy  ##
            self.env = simpy.Environment()
            self.atm = simpy.Resource(self.env, capacity= self.num_atm)
            self.cashier = simpy.Resource(self.env, capacity= self.num_cashier)

        ## process metrics ##
        #
            self.flow_time = []
            self.total_wait_time = []
            self.wait_time_atm = []
            self.wait_time_cashier = []
            self.arrived_customers = 0
            self.finished_customers = 0

        def customer(self, arrival_time):
            '''
            Model the customer flow process
            '''
            #0.5 arrive at the atm
            if np.random.rand() <= 0.5:
                with self.atm.request() as atm_request:
                    yield atm_request
                    running_wait_time = self.env.now - arrival_time
                    self.wait_time_atm.append(running_wait_time)
                    yield self.env.timeout(np.random.triangular(1,2,4))
                    #0.3 of which get to cashier afterwards
                    if np.random.rand() <= 0.3:
                        self.env.process(self.use_cashier(arrival_time, running_wait_time))
                    else:
                    #others finish the process
                        self.total_wait_time.append(running_wait_time)
                        self.finished_customers += 1
                        self.flow_time.append(self.env.now - arrival_time)
            #0.5 go straight to the cashier
            else:
                running_wait_time = 0
                self.env.process(self.use_cashier(arrival_time, running_wait_time))

        def use_cashier(self, arrival_time, running_wait_time):
            """
            Simulates a customer going to cashier service
            """
            cashier_arrival_time = self.env.now
            with self.cashier.request() as cashier_request:
                yield cashier_request
                cashier_wait_time = self.env.now - cashier_arrival_time
                self.wait_time_cashier.append(cashier_wait_time)
                self.total_wait_time.append(running_wait_time + cashier_wait_time)
                yield self.env.timeout(np.random.triangular(3,5,20))
                self.finished_customers += 1
                self.flow_time.append(self.env.now - arrival_time)

        def gen_arrivals(self):
            """
            Customer generation process for arrivals
            """
            while True:
                self.arrived_customers += 1
                self.env.process(self.customer(self.env.now))
                yield self.env.timeout(np.random.exponential(1/self.arrival_rate))

        def simulate(self):
            """
            Runs the simulation
            """
            self.env.process(self.gen_arrivals())
            self.env.run(until=self.sim_time)

    # total simulation time in minutes
    sim_time = 10 * 60

    arrival_rate = lam 

    # Instantiate the class and run it
    system = BankSimulation(sim_time, arrival_rate) 
    system.simulate()
    return np.mean(system.flow_time)
